In [1]:
# general functions:
def downsample(time_series,res = '0.2S'):
    # downsamples time_series pandas data frame 
    # time_series: a pandas data.frame for which there is a column 'timestamps' with numeric timestamps
    # res: desired resolution in time after downsampling
    Nvalues = len(time_series.index)
    samplerate = 1/ ((time_series.timestamp[Nvalues-1] - time_series.timestamp[0]) / Nvalues)
    timestart = dt.datetime(1970, 1, 1, 0, 0, 0, 0) #dt.datetime.now()
    start = pd.Timestamp(timestart)
    end = pd.Timestamp(timestart + dt.timedelta(seconds=Nvalues/samplerate))
    t = np.linspace(start.value, end.value, Nvalues)
    t = pd.to_datetime(t)
    time_series['time'] = t
    time_series = time_series.resample(res,on='time').mean() # downsample to 0.2 second intervals
    time_series.index.name = 'time'
    time_series.reset_index(inplace=True)
    return time_series

# Merge features from various modalities into single database

## 1. Load feature data from Video

Load OpenFace output data and load features in pandas dataframe: So far copied from: https://github.com/emrecdem/exploface/blob/master/TUTORIALS/tutorial1.ipynb

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import exploface
import datetime as dt
import numpy as np

In [88]:
exploface.__version__

'0.0.0.dev6'

In [89]:
# specify some paths
emrecdemStudyDataFolder = "/media/sf_sharedfolder/Emotion/emrecdemstudydata"
openface_outputfolder = emrecdemStudyDataFolder + "/OpenFaceOutput"

In [90]:
# Search for files with csv extension, because we are only interested in those
# This assumes that there are no other csv files in folder other than the ones produced by OpenFace.
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

filenames = find_csv_filenames(openface_outputfolder)

In [91]:
filenames

['P18_S2_IAPS_HAPPY_Cfront.csv',
 'P18_S2_IAPS_SAD_Cfront.csv',
 'P21_S2_IAPS_HAPPY_C1.csv',
 'P21_S2_IAPS_SAD_C1.csv']

In [92]:
# select one file to process (in the future this can be a loop over all the files)
openface_file = openface_outputfolder + '/' + filenames[0]
openface_features = exploface.get_feature_time_series(openface_file)

In [93]:
openface_features.head(5)

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.00,0.98,1,0.028242,0.015028,-0.999488,0.045799,-0.019971,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,2,0,0.04,0.98,1,0.012176,-0.017202,-0.999778,0.035786,-0.044246,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,3,0,0.08,0.98,1,0.003201,0.007913,-0.999964,0.030401,-0.016860,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,4,0,0.12,0.98,1,0.011002,0.017785,-0.999781,-0.066090,-0.075202,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
4,5,0,0.16,0.98,1,0.060468,0.055646,-0.996618,-0.093641,0.164191,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [94]:
openface_features.shape


(5036, 714)

In [95]:
openface_features = downsample(openface_features,res = '0.2S')

In [96]:
openface_features.head()

,time,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1970-01-01 00:00:00.000,3.0,0.0,0.08,0.98,1.0,0.023018,0.015834,-0.999126,-0.009549,...,0.0,0.6,1.0,1.0,1.0,0.8,1.0,0.4,0.0,0.0
1,1970-01-01 00:00:00.200,8.0,0.0,0.28,0.98,1.0,0.004188,-0.004074,-0.999607,-0.044963,...,0.0,1.0,1.0,1.0,1.0,1.0,0.6,0.0,0.0,0.0
2,1970-01-01 00:00:00.400,13.0,0.0,0.48,0.97,1.0,-0.010651,-0.061186,-0.997819,0.028500,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1970-01-01 00:00:00.600,18.0,0.0,0.68,0.98,1.0,0.008494,-0.023006,-0.998510,-0.077739,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.8
4,1970-01-01 00:00:00.800,23.0,0.0,0.88,0.97,1.0,0.002077,-0.092899,-0.995490,0.039646,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [97]:
filter_col = [col for col in openface_features if col.startswith('AU')]

In [98]:
type(filter_col)

list

In [99]:
filter_col.insert(0,'time')

In [100]:
filter_col.insert(0,'participant_id')
filter_col.insert(0,'experiment_id')
openface_features['participant_id'] = 1 #dummy value for now
openface_features['experiment_id'] = 1 #dummy value for now

In [101]:
openface_features = openface_features[filter_col]

In [102]:
openface_features.head()

,experiment_id,participant_id,time,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,1,1970-01-01 00:00:00.000,0.0,0.0,0.986,0.0,0.606,0.016,0.020,...,0.0,0.6,1.0,1.0,1.0,0.8,1.0,0.4,0.0,0.0
1,1,1,1970-01-01 00:00:00.200,0.0,0.0,1.070,0.0,0.568,0.114,0.028,...,0.0,1.0,1.0,1.0,1.0,1.0,0.6,0.0,0.0,0.0
2,1,1,1970-01-01 00:00:00.400,0.0,0.0,1.278,0.0,0.246,0.000,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1,1970-01-01 00:00:00.600,0.0,0.0,1.132,0.0,0.604,0.204,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.8
4,1,1,1970-01-01 00:00:00.800,0.0,0.0,1.336,0.0,0.074,0.102,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [105]:
openface_features.columns = openface_features.columns.str.replace('_', '')
openface_features = openface_features.rename(columns = {'experimentid':'experiment_id'})
openface_features = openface_features.rename(columns = {'participantid':'participant_id'})

In [106]:
openface_features.head()

,experiment_id,participant_id,time,AU01r,AU02r,AU04r,AU05r,AU06r,AU07r,AU09r,...,AU12c,AU14c,AU15c,AU17c,AU20c,AU23c,AU25c,AU26c,AU28c,AU45c
0,1,1,1970-01-01 00:00:00.000,0.0,0.0,0.986,0.0,0.606,0.016,0.020,...,0.0,0.6,1.0,1.0,1.0,0.8,1.0,0.4,0.0,0.0
1,1,1,1970-01-01 00:00:00.200,0.0,0.0,1.070,0.0,0.568,0.114,0.028,...,0.0,1.0,1.0,1.0,1.0,1.0,0.6,0.0,0.0,0.0
2,1,1,1970-01-01 00:00:00.400,0.0,0.0,1.278,0.0,0.246,0.000,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1,1970-01-01 00:00:00.600,0.0,0.0,1.132,0.0,0.604,0.204,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.8
4,1,1,1970-01-01 00:00:00.800,0.0,0.0,1.336,0.0,0.074,0.102,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


## 2. Load feature data from audio

Process data with Librosa in pandas dataframe: So far copied from: https://github.com/emrecdem/explibrosa/blob/master/TUTORIALS/tutorial1.ipynb

In [107]:
import matplotlib.pyplot as plt
import os
import explibrosa

In [108]:
explibrosa.__version__

'0.0.0.dev1'

Find wav file that matches the csv file produced by OpenFace based on the assumption that filenames are identical except from file extension

In [109]:
transformfilename = filenames[0]

In [110]:
transformfilename

'P18_S2_IAPS_HAPPY_Cfront.csv'

In [111]:
audiofile_name = transformfilename.replace('.csv','.wav').replace('_Cfront','').replace('_Cside','')

In [112]:
audiofile_name 

'P18_S2_IAPS_HAPPY.wav'

In [113]:
import subprocess
 
# Set up find command
findCMD = 'find ' + emrecdemStudyDataFolder + ' -name ' + audiofile_name 
out = subprocess.Popen(findCMD,shell=True,stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)
# Get standard out and error
(stdout, stderr) = out.communicate()
 
# Save found files to list
filelist = stdout.decode().split()

In [114]:
audiofiles_fullPaths = filelist # probably the list has only one filename
audio_file = audiofiles_fullPaths[0]

In [115]:
os.path.isfile(audio_file)

True

In [116]:
explibrosa.get_info(audio_file)

{'#frames': 3223486, 'duration (min)': 3.4, 'Sample freq (kHz)': 16.0}

In [117]:

time_series = explibrosa.get_feature_time_series(audio_file)

Running librosa (no results found on disk)
RMS energy
     0.3 seconds
Zero crossing
     0.36 seconds
Pitches
     3.29 seconds
  Pitches smoothing
     3.43 seconds
TOTAL execution time: 0.06 min


In [118]:
time_series.head()

,timestamp,rmse,zrc,pitch
0,0.000000,0.013185,0.038095,186.620880
1,0.010000,0.012107,0.066667,185.620404
2,0.020001,0.010913,0.052381,184.326650
3,0.030001,0.011290,0.052381,182.657746
4,0.040002,0.010963,0.042857,180.606069


In [119]:
type(time_series)

pandas.core.frame.DataFrame

### Downsample the time_series pandas dataframe

In [120]:
# Convert timestamps to datetime format in order to be able to do resampe

In [121]:
time_series.head()

,timestamp,rmse,zrc,pitch
0,0.000000,0.013185,0.038095,186.620880
1,0.010000,0.012107,0.066667,185.620404
2,0.020001,0.010913,0.052381,184.326650
3,0.030001,0.011290,0.052381,182.657746
4,0.040002,0.010963,0.042857,180.606069


In [122]:
time_series = downsample(time_series,res = '0.2S')

In [123]:
time_series.head()

,time,timestamp,rmse,zrc,pitch
0,1970-01-01 00:00:00.000,0.095005,0.008786,0.118095,271.211606
1,1970-01-01 00:00:00.200,0.295015,0.008010,0.141905,260.677619
2,1970-01-01 00:00:00.400,0.495025,0.003346,0.186429,239.593368
3,1970-01-01 00:00:00.600,0.695034,0.010654,0.176190,205.652267
4,1970-01-01 00:00:00.800,0.895044,0.020479,0.153571,349.661769


### Add columns and tidy up column names

In [124]:
time_series['participant_id'] = 1 #dummy value for now
time_series['experiment_id'] = 1 #dummy value for now
time_series = time_series.rename(columns={'zrc': 'zcrate'})
time_series.drop(['timestamp'], axis = 1, inplace = True, errors = 'ignore')
time_series = time_series[['participant_id','experiment_id','time','pitch','rmse','zcrate']]

In [125]:
time_series.head()

,participant_id,experiment_id,time,pitch,rmse,zcrate
0,1,1,1970-01-01 00:00:00.000,271.211606,0.008786,0.118095
1,1,1,1970-01-01 00:00:00.200,260.677619,0.008010,0.141905
2,1,1,1970-01-01 00:00:00.400,239.593368,0.003346,0.186429
3,1,1,1970-01-01 00:00:00.600,205.652267,0.010654,0.176190
4,1,1,1970-01-01 00:00:00.800,349.661769,0.020479,0.153571


## 3. Store in database


http://www.sqlitetutorial.net/sqlite-python/create-tables/

In [126]:
import sqlite3

In [127]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [128]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [129]:
def main():
    # Note time in text datatype, which will facilitate ISO8601 "YYYY-MM-DD HH:MM:SS.SSS"
    database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"
     
    sql_create_participants_table = """ CREATE TABLE IF NOT EXISTS participants (
                                        id integer PRIMARY KEY,
                                        age real
                                    ); """
    
    sql_create_experiments_table = """ CREATE TABLE IF NOT EXISTS experiments (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
    sql_create_videofeatures_table = """ CREATE TABLE IF NOT EXISTS videofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id integer NOT NULL,
                                        time text,
                                        AU01r real, AU02r real, AU04r real,
                                        AU05r real, AU06r real, AU07r real,
                                        AU09r real, AU10r real, AU12r real,
                                        AU14r real, AU15r real, AU17r real,
                                        AU20r real, AU23r real, AU25r real,
                                        AU26r real, AU45r real, AU01c real,
                                        AU02c real, AU04c real, AU05c real,
                                        AU06c real, AU07c real, AU09c real,
                                        AU10c real, AU12c real, AU14c real,
                                        AU15c real, AU17c real, AU20c real,
                                        AU23c real, AU25c real, AU26c real,
                                        AU28c real, AU45c real,                                        
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
    
    sql_create_audiofeatures_table = """ CREATE TABLE IF NOT EXISTS audiofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id integer NOT NULL,
                                        time text, 
                                        pitch real,
                                        rmse real,
                                        zcrate real,
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
        
    sql_create_facslabels_table = """ CREATE TABLE IF NOT EXISTS facslabels (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
 
    sql_create_facsencoding_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    participant_id integer NOT NULL,
                                    experiment_id integer NOT NULL,
                                    timestamp text,
                                    emotion_id integer NOT NULL,
                                    FOREIGN KEY (participant_id) REFERENCES participants (id),
                                    FOREIGN KEY (experiment_id) REFERENCES experiments (id),
                                    FOREIGN KEY (emotion_id) REFERENCES facslabels (id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create participants table
        create_table(conn, sql_create_participants_table)
        # create experiments table
        create_table(conn, sql_create_experiments_table)
        create_table(conn, sql_create_videofeatures_table)
        create_table(conn, sql_create_audiofeatures_table)
        create_table(conn, sql_create_facslabels_table)
        create_table(conn, sql_create_facsencoding_table)
    else:
        print("Error! cannot create the database connection.")

In [130]:

if __name__ == '__main__':
    main()

In [131]:
# Now on command line: sqlite3 pathtomydatabase.db
# should give access to database
# and .tables should show overview of the tables as created

### To do Video:
- Strip away irrelevant information, e.g. make selection of features of interest.

### To do Video + Audio:
- Retrieve participant ID from files and store these together with data and filename in database.
- Develop demo for searchability


In [132]:
#https://www.dataquest.io/blog/python-pandas-databases/
database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"

In [133]:
conn = create_connection(database) # create connection
cur = conn.cursor() # create cursor

### Add Audio features to database

In [134]:
# delete contents of table for testing purposes, later we obviously do not want to do this
rowcount = cur.execute("DELETE FROM audiofeatures;").fetchall()

In [135]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [136]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])

Top 5 rows of audiofeatures table:
[]

Row count:
0


In [137]:
print("\nColumn names:")
print(colnames)


Column names:
[(0, 'id', 'integer', 0, None, 1), (1, 'participant_id', 'integer', 1, None, 0), (2, 'experiment_id', 'integer', 1, None, 0), (3, 'time', 'text', 0, None, 0), (4, 'pitch', 'real', 0, None, 0), (5, 'rmse', 'real', 0, None, 0), (6, 'zcrate', 'real', 0, None, 0)]


In [138]:
# move time_series pandas data frame to the database
time_series.to_sql("audiofeatures", conn, index= False, if_exists="append") #"replace"

In [139]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [140]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)

Top 5 rows of audiofeatures table:
[(1, 1, 1, '1970-01-01 00:00:00', 271.2116057159737, 0.008785701356828213, 0.1180952380952381), (2, 1, 1, '1970-01-01 00:00:00.200000', 260.67761899075373, 0.008010043762624264, 0.1419047619047619), (3, 1, 1, '1970-01-01 00:00:00.400000', 239.59336760187193, 0.0033461761195212603, 0.18642857142857144), (4, 1, 1, '1970-01-01 00:00:00.600000', 205.65226665554346, 0.010653898119926453, 0.17619047619047618), (5, 1, 1, '1970-01-01 00:00:00.800000', 349.66176893618893, 0.02047872543334961, 0.15357142857142855)]

Row count:
1008

Dimensions of pandas dataframe:
(1008, 6)


In [141]:
print("\nColumn names:")
print(colnames)


Column names:
[(0, 'id', 'integer', 0, None, 1), (1, 'participant_id', 'integer', 1, None, 0), (2, 'experiment_id', 'integer', 1, None, 0), (3, 'time', 'text', 0, None, 0), (4, 'pitch', 'real', 0, None, 0), (5, 'rmse', 'real', 0, None, 0), (6, 'zcrate', 'real', 0, None, 0)]


### Add video data to database

In [142]:
# delete contents of table for testing purposes, later we obviously do not want to do this
rowcount = cur.execute("DELETE FROM videofeatures;").fetchall()

In [143]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(videofeatures);").fetchall()

In [144]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])

Top 5 rows of videofeatures table:
[]

Row count:
0


In [145]:
print("\nColumn names:")
print(colnames)


Column names:
[(0, 'id', 'integer', 0, None, 1), (1, 'participant_id', 'integer', 1, None, 0), (2, 'experiment_id', 'integer', 1, None, 0), (3, 'time', 'text', 0, None, 0), (4, 'AU01r', 'real', 0, None, 0), (5, 'AU02r', 'real', 0, None, 0), (6, 'AU04r', 'real', 0, None, 0), (7, 'AU05r', 'real', 0, None, 0), (8, 'AU06r', 'real', 0, None, 0), (9, 'AU07r', 'real', 0, None, 0), (10, 'AU09r', 'real', 0, None, 0), (11, 'AU10r', 'real', 0, None, 0), (12, 'AU12r', 'real', 0, None, 0), (13, 'AU14r', 'real', 0, None, 0), (14, 'AU15r', 'real', 0, None, 0), (15, 'AU17r', 'real', 0, None, 0), (16, 'AU20r', 'real', 0, None, 0), (17, 'AU23r', 'real', 0, None, 0), (18, 'AU25r', 'real', 0, None, 0), (19, 'AU26r', 'real', 0, None, 0), (20, 'AU45r', 'real', 0, None, 0), (21, 'AU01c', 'real', 0, None, 0), (22, 'AU02c', 'real', 0, None, 0), (23, 'AU04c', 'real', 0, None, 0), (24, 'AU05c', 'real', 0, None, 0), (25, 'AU06c', 'real', 0, None, 0), (26, 'AU07c', 'real', 0, None, 0), (27, 'AU09c', 'real', 0, No

In [146]:
openface_features.head()

,experiment_id,participant_id,time,AU01r,AU02r,AU04r,AU05r,AU06r,AU07r,AU09r,...,AU12c,AU14c,AU15c,AU17c,AU20c,AU23c,AU25c,AU26c,AU28c,AU45c
0,1,1,1970-01-01 00:00:00.000,0.0,0.0,0.986,0.0,0.606,0.016,0.020,...,0.0,0.6,1.0,1.0,1.0,0.8,1.0,0.4,0.0,0.0
1,1,1,1970-01-01 00:00:00.200,0.0,0.0,1.070,0.0,0.568,0.114,0.028,...,0.0,1.0,1.0,1.0,1.0,1.0,0.6,0.0,0.0,0.0
2,1,1,1970-01-01 00:00:00.400,0.0,0.0,1.278,0.0,0.246,0.000,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1,1970-01-01 00:00:00.600,0.0,0.0,1.132,0.0,0.604,0.204,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.8
4,1,1,1970-01-01 00:00:00.800,0.0,0.0,1.336,0.0,0.074,0.102,0.000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [147]:
# move time_series pandas data frame to the database
openface_features.to_sql("videofeatures", conn, index=False,if_exists="append") #"replace"

In [148]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()

In [149]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)

Top 5 rows of videofeatures table:
[(1, 1, 1, '1970-01-01 00:00:00', 0.0, 0.0, 0.986, 0.0, 0.606, 0.016, 0.02, 0.916, 0.6340000000000001, 1.1380000000000001, 0.44400000000000006, 1.054, 0.516, 0.8, 0.9039999999999999, 2.7339999999999995, 0.22599999999999998, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 0.6, 0.0, 0.6, 1.0, 1.0, 1.0, 0.8, 1.0, 0.4, 0.0, 0.0), (2, 1, 1, '1970-01-01 00:00:00.200000', 0.0, 0.0, 1.07, 0.0, 0.568, 0.11400000000000002, 0.028000000000000004, 1.346, 0.642, 1.5260000000000002, 0.37399999999999994, 1.206, 0.442, 0.8480000000000001, 0.30199999999999994, 2.8240000000000003, 0.21799999999999997, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 0.0, 0.0, 0.0), (3, 1, 1, '1970-01-01 00:00:00.400000', 0.0, 0.0, 1.278, 0.0, 0.246, 0.0, 0.0, 1.214, 0.45, 1.5699999999999998, 0.384, 1.658, 0.41600000000000004, 1.766, 0.008, 2.02, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0), (4, 1, 1, '1970-01-01 00:00:00.600

In [150]:
cur.close()
conn.close()